после уточнения вертикальных осей в dgn мы экспортируем их как набор точек

задача обновить начальную таблицу с координатами опор

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import csv
from math import sqrt, acos, degrees

In [38]:
# пути
workdir = Path(r'D:\work\2022_m\220_man-van\test')

In [39]:
# читаем начальные координаты из полученного ранее файла
cgt = pd.read_excel(workdir / 'cgtw_output.xlsx', usecols='A:F', index_col=0)

# старая версия из ctow
# cgt = pd.read_csv(workdir / '220_ctow.txt', delimiter='\t', header=None, names = ['id', 'x', 'y', 'z'], encoding = 'cp1251')

In [40]:
cgt

,c_line,id,x,y,z
index,,,,,
1,1,ПС_Мангазея,557351.89,7388273.90,0
2,1,1,557342.11,7388234.12,0
3,1,2,557410.59,7388150.65,0
4,1,3,557529.27,7388004.57,0
5,1,4,557662.96,7388000.01,0
...,...,...,...,...,...
605,1,604,604094.27,7513440.66,0
606,1,605,604019.26,7513536.77,0
607,1,606,603962.89,7513557.05,0


In [41]:
# лист уточненныхъ координат (без дублирования)
list_corr = []
with open(workdir / '220_cor.txt', 'r') as f:
    for line in f:
        list_corr.append(line.split())

In [42]:
temp_id_list = []   # list to add used id
num_of_strokes = 0   # add one if add to list

# add lists with data for new colums
x_bot = []
y_bot = []
z_bot = []
x_top = []
y_top = []
z_top = []


for i in range(1, len(cgt)+1):
    if cgt['id'][i] not in temp_id_list:
        temp_id_list.append(cgt['id'][i])
        x_bot.append(float(list_corr[num_of_strokes*2][0]))
        y_bot.append(float(list_corr[num_of_strokes*2][1]))
        z_bot.append(float(list_corr[num_of_strokes*2][2]))
        x_top.append(float(list_corr[num_of_strokes*2+1][0]))
        y_top.append(float(list_corr[num_of_strokes*2+1][1]))
        z_top.append(float(list_corr[num_of_strokes*2+1][2]))
        num_of_strokes += 1
    else:
        temp_i = cgt.index[cgt['id'] == cgt['id'][i]].tolist()[0]-1    # change index if this number was before
        # copy from previous
        x_bot.append(float(list_corr[temp_i*2][0]))
        y_bot.append(float(list_corr[temp_i*2][1]))
        z_bot.append(float(list_corr[temp_i*2][2]))
        x_top.append(float(list_corr[temp_i*2+1][0]))
        y_top.append(float(list_corr[temp_i*2+1][1]))
        z_top.append(float(list_corr[temp_i*2+1][2]))

# then add columns to the table 
cgt.insert(5, 'x_bot', x_bot)
cgt.insert(6, 'y_bot', y_bot)
cgt.insert(7, 'z_bot', z_bot)
cgt.insert(8, 'x_top', x_top)
cgt.insert(9, 'y_top', y_top)
cgt.insert(10, 'z_top', z_top)

In [43]:
cgt

,c_line,id,x,y,z,x_bot,y_bot,z_bot,x_top,y_top,z_top
index,,,,,,,,,,,
1,1,ПС_Мангазея,557351.89,7388273.90,0,557349.94,7388274.42,19.52,557349.99,7388274.32,43.20
2,1,1,557342.11,7388234.12,0,557342.11,7388234.95,17.89,557342.24,7388234.82,56.03
3,1,2,557410.59,7388150.65,0,557409.02,7388151.95,16.39,557409.07,7388151.96,58.47
4,1,3,557529.27,7388004.57,0,557526.72,7388005.93,15.58,557526.87,7388006.06,57.53
5,1,4,557662.96,7388000.01,0,557662.46,7388000.73,18.41,557662.47,7388000.81,65.46
...,...,...,...,...,...,...,...,...,...,...,...
605,1,604,604094.27,7513440.66,0,604092.15,7513441.75,33.82,604091.19,7513441.49,71.97
606,1,605,604019.26,7513536.77,0,604017.81,7513538.14,32.59,604017.69,7513538.15,70.69
607,1,606,603962.89,7513557.05,0,603964.49,7513556.95,33.74,603964.45,7513556.80,71.75


In [44]:
def distance(a, b):
    print(a)
    print(a[0])
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dist = round(sqrt(dx*dx + dy*dy), 2)    # dist a to b
    return dist

In [45]:
cgt['dist'] = cgt.apply(lambda row: distance((row.x, row.y), (row.x_bot, row.y_bot)), axis = 1)

(557351.89, 7388273.9)
557351.89
(557342.11, 7388234.12)
557342.11
(557410.59, 7388150.65)
557410.59
(557529.27, 7388004.57)
557529.27
(557662.96, 7388000.01)
557662.96
(557811.32, 7387994.46)
557811.32
(557948.27, 7387990.88)
557948.27
(558084.57, 7388194.99)
558084.57
(558163.47, 7388310.42)
558163.47
(558202.6, 7388370.09)
558202.6
(558571.38, 7388289.55)
558571.38
(558874.95, 7388224.67)
558874.95
(559216.02, 7388150.32)
559216.02
(559540.13, 7388082.17)
559540.13
(559862.28, 7388012.72)
559862.28
(560185.09, 7387944.57)
560185.09
(560478.55, 7387882.29)
560478.55
(560791.58, 7387814.8)
560791.58
(561114.38, 7387746.0)
561114.38
(561442.41, 7387676.55)
561442.41
(561771.73, 7387604.49)
561771.73
(562146.06, 7387526.23)
562146.06
(562511.25, 7387447.65)
562511.25
(562792.32, 7387387.98)
562792.32
(563038.18, 7387281.35)
563038.18
(563314.68, 7387162.34)
563314.68
(563640.75, 7387021.15)
563640.75
(563951.17, 7386885.83)
563951.17
(564257.67, 7386752.47)
564257.67
(564558.96, 7386621

(585676.55, 7445177.82)
585676.55
(585756.43, 7445426.81)
585756.43
(585816.45, 7445616.8)
585816.45
(585915.81, 7445933.1)
585915.81
(586007.45, 7446221.83)
586007.45
(586099.1, 7446514.62)
586099.1
(586191.15, 7446808.61)
586191.15
(586285.23, 7447103.83)
586285.23
(586363.49, 7447356.88)
586363.49
(586435.27, 7447585.99)
586435.27
(586522.85, 7447862.15)
586522.85
(586608.42, 7448136.69)
586608.42
(586699.67, 7448420.15)
586699.67
(586770.63, 7448649.07)
586770.63
(586843.22, 7448876.97)
586843.22
(586915.0, 7449111.76)
586915.0
(586991.24, 7449349.2)
586991.24
(587066.67, 7449588.45)
587066.67
(587142.5, 7449829.33)
587142.5
(587221.17, 7450078.53)
587221.17
(587301.46, 7450330.76)
587301.46
(587382.56, 7450587.87)
587382.56
(587464.47, 7450846.19)
587464.47
(587545.58, 7451108.56)
587545.58
(587630.74, 7451374.58)
587630.74
(587724.42, 7451675.07)
587724.42
(587815.25, 7451965.83)
587815.25
(587903.66, 7452248.08)
587903.66
(587972.2, 7452462.8)
587972.2
(588082.49, 7452810.94)
58

In [209]:
cgt2 = cgt[cgt['dist'] > 2]

In [210]:
cgt2

,id,x,y,z,x_bot,y_bot,z_bot,x_top,y_top,z_top,dist
0,ПС_Мангазея,557351.89,7388273.90,0.0,557349.94,7388274.42,19.52,557349.99,7388274.32,43.20,2.02
2,2,557410.59,7388150.65,0.0,557409.02,7388151.95,16.39,557409.07,7388151.96,58.47,2.04
3,3,557529.27,7388004.57,0.0,557526.72,7388005.93,15.58,557526.87,7388006.06,57.53,2.89
7,7,558084.57,7388194.99,0.0,558083.86,7388193.00,19.39,558083.85,7388192.98,59.01,2.11
9,9,558202.60,7388370.09,0.0,558200.93,7388368.58,18.85,558201.27,7388368.55,60.87,2.25
28,28,564257.67,7386752.47,0.0,564255.66,7386753.92,30.24,564255.69,7386753.82,70.15,2.48
30,30,564896.76,7386474.66,0.0,564895.75,7386476.55,29.95,564895.98,7386475.03,75.13,2.14
38,38,567221.62,7385442.33,0.0,567221.39,7385445.07,32.22,567221.10,7385443.14,72.25,2.75
39,39,567528.12,7385305.71,0.0,567526.58,7385307.18,31.67,567526.86,7385307.62,71.39,2.13
41,41,568130.04,7385035.73,0.0,568127.35,7385036.01,31.89,568127.36,7385036.05,71.77,2.70


In [46]:
cgt.to_csv(workdir / '220_v2.txt', sep='\t', encoding = 'cp1251')